In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function, division

from matplotlib import pyplot as plt
import pandas
import src
import gensim
import os
import os.path
import csv
import functools
import itertools
import collections
import scipy
import scipy.stats
from operator import itemgetter

In [3]:
model_config = {
    'num_topics': 500,
    'alpha': 1/500,
    'eta': 1/500,
    'alpha_base': 1,
    'eta_base': 1,
    'decay': 0.5,
    'offset': 1.0,
    'iterations': 1000,
    'passes': 1,
    'max_bound_iterations': 1000, # special
    'algorithm': 'batch', # special
}

changeset_config = {
    'include_additions': True,
    'include_context': True,
    'include_message': False,
    'include_removals': True,
}

def get_config_string(config):
    return '-'.join([unicode(v) for k, v in sorted(config.items())])

alpha_bases = ['auto', 1, 2, 5]
eta_bases = ['auto', 1, 2, 5]
num_topics = [100, 200, 500]

def get_rank_name(kind, experiment, changeset_config, model_config):
    """
    kind = [changeset, release, temporal]
    experiment = [triage, feature_location]
    """
    cs_str = get_config_string(changeset_config)
    model_config = dict(model_config)
    del model_config['alpha_base']
    del model_config['eta_base']
    m_str = get_config_string(model_config)
    return '-'.join([kind, experiment, 'lda', cs_str, m_str, 'file', 'ranks']).lower() + '.csv.gz'

model_sweep = list()
for a, e, K in itertools.product(alpha_bases, eta_bases, num_topics):
    m = dict(model_config)
    m['alpha_base'] = a
    m['eta_base'] = e

    if a != 'auto':
        a /= K
    if e != 'auto':
        e /= K
    m['alpha'] = a
    m['eta'] = e
    m['num_topics'] = K
    model_sweep.append(m)

corpus_sweep = list()
b = [True, False]
for a, c, m, r in itertools.product(b, repeat=4):
    conf = dict(changeset_config)
    conf['include_additions'] = a
    conf['include_context'] = c
    conf['include_message'] = m
    conf['include_removals'] = r
    if any(conf.values()):
        corpus_sweep.append(conf)

In [18]:
projects = src.common.load_projects(dict(model="lda", level="file", rankpath='', config=dict()), "../data")
projects

[Project(name='tika', printable_name='Tika v1.8', version='v1.8', ref='refs/tags/1.8', data_path='../data/tika/', full_path='../data/tika/v1.8/', src_path='../data/tika/v1.8/src/', model='lda', rankpath='', config={}, level='file'),
 Project(name='pig', printable_name='Pig v0.14.0', version='v0.14.0', ref='refs/tags/release-0.14.0', data_path='../data/pig/', full_path='../data/pig/v0.14.0/', src_path='../data/pig/v0.14.0/src/', model='lda', rankpath='', config={}, level='file'),
 Project(name='bookkeeper', printable_name='BookKeeper v4.3.0', version='v4.3.0', ref='refs/tags/release-4.3.0', data_path='../data/bookkeeper/', full_path='../data/bookkeeper/v4.3.0/', src_path='../data/bookkeeper/v4.3.0/src/', model='lda', rankpath='', config={}, level='file'),
 Project(name='openjpa', printable_name='OpenJPA v2.3.0', version='v2.3.0', ref='refs/tags/2.3.0', data_path='../data/openjpa/', full_path='../data/openjpa/v2.3.0/', src_path='../data/openjpa/v2.3.0/src/', model='lda', rankpath='', con

In [22]:
cs_dit = list()
for project, rankstuff in itertools.product(projects, [
        (c, get_rank_name('changeset', 'triage', c, model_config)) for c in corpus_sweep]):
    config, rankname = rankstuff
    rankpath = os.path.join(project.data_path, project.version, rankname)
    if os.path.exists(rankpath):
        cs_dit.append(project._replace(rankpath=rankpath, config=config))
    else:
        print(rankpath)

cs_flt = list()
for project, rankstuff in itertools.product(projects, [
        (c, get_rank_name('changeset', 'feature_location', c, model_config)) for c in corpus_sweep]):
    config, rankname = rankstuff
    rankpath = os.path.join(project.data_path, project.version,  rankname)
    if os.path.exists(rankpath):
        cs_flt.append(project._replace(rankpath=rankpath, config=config))
    else:
        print(rankpath)

        
ms_dit = list()
for project, rankstuff in itertools.product(projects, [
        (c, get_rank_name('changeset', 'triage', changeset_config, c)) for c in model_sweep]):
    config, rankname = rankstuff
    rankpath = os.path.join(project.data_path, project.version,  rankname)
    if os.path.exists(rankpath):
        ms_dit.append(project._replace(rankpath=rankpath, config=config))
    else:
        print(rankpath)
        
ms_flt = list()
for project, rankstuff in itertools.product(projects, [
        (c, get_rank_name('changeset', 'feature_location', changeset_config, c)) for c in model_sweep]):
    config, rankname = rankstuff
    rankpath = os.path.join(project.data_path, project.version,  rankname)
    if os.path.exists(rankpath):
         ms_flt.append(project._replace(rankpath=rankpath, config=config))
    else:
        print(rankpath)

In [23]:
corpus_df = pandas.DataFrame(columns=["Subject", "Task","Issue", "Rank", "Distance", "Additions", "Removals", "Context", "Message"])
for item in cs_dit:
    df = pandas.DataFrame(columns=corpus_df.columns)
    subdf = pandas.read_csv(item.rankpath)
    df["Rank"] = subdf.groupby("id")["rank"].min().values
    df["Distance"] = subdf.groupby("id")["distance"].min().values
    df["Issue"] = subdf.groupby("id")["rank"].min().index
    df["Subject"] = df.Subject.fillna(item.printable_name)
    df["Additions"] = df.Additions.fillna(item.config['include_additions'])
    df["Removals"] = df.Removals.fillna(item.config['include_removals'])
    df["Context"] = df.Context.fillna(item.config['include_context'])
    df["Message"] = df.Message.fillna(item.config['include_message'])
    df["Task"] = df.Task.fillna("DIT")
    corpus_df = corpus_df.append(df, ignore_index=True)
    
for item in cs_flt:
    df = pandas.DataFrame(columns=corpus_df.columns)
    subdf = pandas.read_csv(item.rankpath)
    df["Rank"] = subdf.groupby("id")["rank"].min().values
    df["Distance"] = subdf.groupby("id")["distance"].min().values
    df["Issue"] = subdf.groupby("id")["rank"].min().index
    df["Subject"] = df.Subject.fillna(item.printable_name)
    df["Additions"] = df.Additions.fillna(item.config['include_additions'])
    df["Removals"] = df.Removals.fillna(item.config['include_removals'])
    df["Context"] = df.Context.fillna(item.config['include_context'])
    df["Message"] = df.Message.fillna(item.config['include_message'])
    df["Task"] = df.Task.fillna("FLT")
    corpus_df = corpus_df.append(df, ignore_index=True)
    
model_df = pandas.DataFrame(columns=["Subject", "Task", "Issue", "Rank", "Distance", "alpha", "eta", "K"])
for item in ms_dit:
    df = pandas.DataFrame(columns=model_df.columns)
    subdf = pandas.read_csv(item.rankpath)
    df["Rank"] = subdf.groupby("id")["rank"].min().values
    df["Distance"] = subdf.groupby("id")["distance"].min().values
    df["Issue"] = subdf.groupby("id")["rank"].min().index
    df["Subject"] = df.Subject.fillna(item.printable_name)
    df["alpha"] = df.alpha.fillna(item.config['alpha_base'])
    df["eta"] = df.eta.fillna(item.config['eta_base'])
    df["K"] = df.K.fillna(item.config['num_topics'])
    df["Task"] = df.Task.fillna("DIT")
    model_df = model_df.append(df, ignore_index=True)
    
for item in ms_flt:
    df = pandas.DataFrame(columns=model_df.columns)
    subdf = pandas.read_csv(item.rankpath)
    df["Rank"] = subdf.groupby("id")["rank"].min().values
    df["Distance"] = subdf.groupby("id")["distance"].min().values
    df["Issue"] = subdf.groupby("id")["rank"].min().index
    df["Subject"] = df.Subject.fillna(item.printable_name)
    df["alpha"] = df.alpha.fillna(item.config['alpha_base'])
    df["eta"] = df.eta.fillna(item.config['eta_base'])
    df["K"] = df.K.fillna(item.config['num_topics'])
    df["Task"] = df.Task.fillna("FLT")
    model_df = model_df.append(df, ignore_index=True)

# Corpus analysis

In [24]:
corpus_df[:10]

,Subject,Task,Issue,Rank,Distance,Additions,Removals,Context,Message
0,Tika v1.8,DIT,241,1,0.848424,True,True,True,True
1,Tika v1.8,DIT,936,6,0.872562,True,True,True,True
2,Tika v1.8,DIT,995,13,0.936484,True,True,True,True
3,Tika v1.8,DIT,1028,1,0.850094,True,True,True,True
4,Tika v1.8,DIT,1269,3,0.854914,True,True,True,True
5,Tika v1.8,DIT,1286,1,0.820050,True,True,True,True
6,Tika v1.8,DIT,1365,10,0.920664,True,True,True,True
7,Tika v1.8,DIT,1383,3,0.818742,True,True,True,True
8,Tika v1.8,DIT,1416,5,0.895160,True,True,True,True
9,Tika v1.8,DIT,1423,2,0.847252,True,True,True,True


In [25]:
(corpus_df.groupby(["Subject", "Task", "Additions"]).Rank.apply(src.utils.calculate_mrr),
 '********************************************',
 corpus_df.groupby(["Subject", "Task", "Removals"]).Rank.apply(src.utils.calculate_mrr),
 '********************************************',
 corpus_df.groupby(["Subject", "Task", "Context"]).Rank.apply(src.utils.calculate_mrr),
 '********************************************',
 corpus_df.groupby(["Subject", "Task", "Message"]).Rank.apply(src.utils.calculate_mrr),
)

(Subject            Task  Additions
 BookKeeper v4.3.0  DIT   False        0.626974
                          True         0.640180
                    FLT   False        0.480202
                          True         0.564261
 Mahout v0.10.0     DIT   False        0.276397
                          True         0.306394
                    FLT   False        0.658185
                          True         0.670980
 OpenJPA v2.3.0     DIT   False        0.326048
                          True         0.355134
                    FLT   False        0.314955
                          True         0.318973
 Pig v0.14.0        DIT   False        0.209869
                          True         0.181139
                    FLT   False        0.449069
                          True         0.481644
 Tika v1.8          DIT   False        0.367664
                          True         0.401152
                    FLT   False        0.484871
                          True         0.565757
 Zoo

In [26]:
res = pandas.DataFrame(columns=["Subject", "Config", "NotIncl", "Incl", "p"])
for k in ["Additions", "Removals", "Context", "Message"]:
    for key, group in corpus_df.groupby(["Subject", "Task"]):
        sub = group.groupby(k).groups
        f = corpus_df.ix[sub[False]].Rank
        t = corpus_df.ix[sub[True]].Rank
        stat, p = scipy.stats.mannwhitneyu(f, t)
        res = res.append(
            dict(
                zip(res.columns, 
                    [key, k, src.utils.calculate_mrr(f), src.utils.calculate_mrr(t), p]))
            , ignore_index=True)
    sub = corpus_df.groupby(["Task", k]).groups
    f = corpus_df.ix[sub[("DIT", False)]].Rank
    t = corpus_df.ix[sub[("DIT", True)]].Rank
    stat, p = scipy.stats.mannwhitneyu(f, t)
    res = res.append(
        dict(
            zip(res.columns, 
                ["**Overall DIT**", k, src.utils.calculate_mrr(f), src.utils.calculate_mrr(t), p]))
        , ignore_index=True)
    f = corpus_df.ix[sub[("FLT", False)]].Rank
    t = corpus_df.ix[sub[("FLT", True)]].Rank
    stat, p = scipy.stats.mannwhitneyu(f, t)
    res = res.append(
        dict(
            zip(res.columns, 
                ["**Overall FLT**", k, src.utils.calculate_mrr(f), src.utils.calculate_mrr(t), p]))
        , ignore_index=True)        

In [27]:
res[(res.NotIncl > res.Incl) & (res.p < 0.05)]

,Subject,Config,NotIncl,Incl,p
6,"(Pig v0.14.0, DIT)",Additions,0.209869,0.181139,6.938320e-03
15,"(BookKeeper v4.3.0, FLT)",Removals,0.544180,0.508280,6.190812e-03
16,"(Mahout v0.10.0, DIT)",Removals,0.301791,0.284174,6.546379e-03
20,"(Pig v0.14.0, DIT)",Removals,0.216694,0.175167,1.711173e-15
24,"(ZooKeeper v3.5.0, DIT)",Removals,0.371838,0.338587,2.010238e-06
26,**Overall DIT**,Removals,0.365642,0.347235,1.851102e-09


In [28]:
res[(res.NotIncl < res.Incl) & (res.p < 0.05)]

,Subject,Config,NotIncl,Incl,p
1,"(BookKeeper v4.3.0, FLT)",Additions,0.480202,0.564261,6.631229e-08
2,"(Mahout v0.10.0, DIT)",Additions,0.276397,0.306394,3.100878e-03
4,"(OpenJPA v2.3.0, DIT)",Additions,0.326048,0.355134,1.685170e-03
7,"(Pig v0.14.0, FLT)",Additions,0.449069,0.481644,9.715458e-03
9,"(Tika v1.8, FLT)",Additions,0.484871,0.565757,1.784296e-02
10,"(ZooKeeper v3.5.0, DIT)",Additions,0.353904,0.354280,8.614995e-03
12,**Overall DIT**,Additions,0.353177,0.358141,2.190292e-03
13,**Overall FLT**,Additions,0.518900,0.551194,3.529204e-06
18,"(OpenJPA v2.3.0, DIT)",Removals,0.329526,0.352091,3.885368e-02
28,"(BookKeeper v4.3.0, DIT)",Context,0.592610,0.670249,1.407148e-07


In [ ]:
# for key, group in corpus_df.groupby(["Subject", "Task"]):
#     ranks = dict()
#     for subkey, subgroup in group.groupby(["Additions", "Removals", "Context", "Message"]):
#         ranks[subkey] = subgroup.Rank

#     print(key, scipy.stats.friedmanchisquare(*ranks.values()))
#     for x, y in itertools.combinations(corpus_df.groupby(["Additions", "Removals", "Context", "Message"]).groups.keys(), r=2):
#         stat, p = scipy.stats.wilcoxon(ranks[x], ranks[y])
#         if p < 0.01:
#             print(x, y, p, "******")
#         else:
#             print(x, y, p)
#     print()

In [29]:
for key, group in corpus_df.groupby(["Task"]):
    ranks = dict()
    for subkey, subgroup in group.groupby(["Additions", "Removals", "Context", "Message"]):
        ranks[subkey] = subgroup.Rank

    print(key, scipy.stats.friedmanchisquare(*ranks.values()))

for key, group in corpus_df.groupby(["Subject", "Task"]):
    ranks = dict()
    for subkey, subgroup in group.groupby(["Additions", "Removals", "Context", "Message"]):
        ranks[subkey] = subgroup.Rank

    print(key, scipy.stats.friedmanchisquare(*ranks.values()))

DIT (865.79254548948757, 9.1739283942016581e-176)
FLT (155.88057495083058, 4.7698405563397853e-26)
('BookKeeper v4.3.0', 'DIT') (233.15631359025716, 8.6269867968642809e-42)
('BookKeeper v4.3.0', 'FLT') (106.98127549672772, 2.1488800121190289e-16)
('Mahout v0.10.0', 'DIT') (63.163414841953234, 3.2463273548226162e-08)
('Mahout v0.10.0', 'FLT') (9.9835062516621615, 0.76338834686468915)
('OpenJPA v2.3.0', 'DIT') (79.634937268117895, 3.3067747040141524e-11)
('OpenJPA v2.3.0', 'FLT') (44.970187585419502, 4.1250518645336056e-05)
('Pig v0.14.0', 'DIT') (689.40014155951508, 4.7160738177830371e-138)
('Pig v0.14.0', 'FLT') (40.276026174897218, 0.00023102075215966668)
('Tika v1.8', 'DIT') (50.65179804593577, 4.7470876363070531e-06)
('Tika v1.8', 'FLT') (38.132266217354761, 0.00049598469521730038)
('ZooKeeper v3.5.0', 'DIT') (371.87543769961064, 1.0502989811473344e-70)
('ZooKeeper v3.5.0', 'FLT') (42.390670195089115, 0.00010716208589086559)


# Model analysis

In [30]:
model_df[:10]

,Subject,Task,Issue,Rank,Distance,alpha,eta,K
0,Tika v1.8,DIT,241,21,0.936748,auto,auto,100
1,Tika v1.8,DIT,936,16,0.926686,auto,auto,100
2,Tika v1.8,DIT,995,17,0.966241,auto,auto,100
3,Tika v1.8,DIT,1028,11,0.832197,auto,auto,100
4,Tika v1.8,DIT,1269,5,0.813074,auto,auto,100
5,Tika v1.8,DIT,1286,4,0.808728,auto,auto,100
6,Tika v1.8,DIT,1365,13,0.870110,auto,auto,100
7,Tika v1.8,DIT,1383,4,0.812833,auto,auto,100
8,Tika v1.8,DIT,1416,13,0.901925,auto,auto,100
9,Tika v1.8,DIT,1423,4,0.830083,auto,auto,100


In [31]:
# for key, group in model_df.groupby(["Subject", "Task"]):
#     ranks = dict()
#     for subkey, subgroup in group.groupby(["alpha", "eta", "K"]):
#         ranks[subkey] = subgroup.Rank

#     print(key, scipy.stats.friedmanchisquare(*ranks.values()))
#     for x, y in itertools.combinations(model_df.groupby(["alpha", "eta", "K"]).groups.keys(), r=2):
#         stat, p = scipy.stats.wilcoxon(ranks[x], ranks[y])
#         if p < 0.01:
#             print(x, y, p, "******")
#         else:
#             print(x, y, p)
#     print()

In [32]:
for key, group in model_df.groupby(["Task"]):
    ranks = dict()
    for subkey, subgroup in group.groupby(["alpha", "eta", "K"]):
        ranks[subkey] = subgroup.Rank

    print(key, scipy.stats.friedmanchisquare(*ranks.values()))

for key, group in model_df.groupby(["Subject", "Task"]):
    ranks = dict()
    for subkey, subgroup in group.groupby(["alpha", "eta", "K"]):
        ranks[subkey] = subgroup.Rank

    print(key, scipy.stats.friedmanchisquare(*ranks.values()))

DIT (2344.7424410923445, 0.0)
FLT (677.52510115606503, 1.2601439840788187e-112)
('BookKeeper v4.3.0', 'DIT') (996.38417304284792, 4.1697686372225943e-178)
('BookKeeper v4.3.0', 'FLT') (78.798391003078535, 0.0025058102159384223)
('Mahout v0.10.0', 'DIT') (333.07562541926177, 9.7923909509142532e-45)
('Mahout v0.10.0', 'FLT') (76.401621763193518, 0.0042906441933269011)
('OpenJPA v2.3.0', 'DIT') (193.71005927121419, 1.0185442311884395e-19)
('OpenJPA v2.3.0', 'FLT') (230.9632412498095, 4.142422491413179e-26)
('Pig v0.14.0', 'DIT') (838.84959423885743, 1.4140219099607691e-145)
('Pig v0.14.0', 'FLT') (293.88490037289029, 1.9355653670223743e-37)
('Tika v1.8', 'DIT') (172.75961213545958, 2.8476468887569913e-16)
('Tika v1.8', 'FLT') (213.41459009922494, 4.6154660020776987e-23)
('ZooKeeper v3.5.0', 'DIT') (1224.1693765618186, 1.4629296010885519e-225)
('ZooKeeper v3.5.0', 'FLT') (144.94961459170079, 6.4161588515107506e-12)


In [33]:
res = pandas.DataFrame(columns=["Subject", "Task", "Config", "Config2", "MRR", "MRR2", "p"])
for k in ["alpha", "eta", "K"]:
    for key, group in model_df.groupby(["Subject", "Task"]):
        ranks = dict()
        for subkey, subgroup in group.groupby(k):
            ranks[subkey] = subgroup.Rank
        
        for each in itertools.combinations(ranks.keys(), r=2):
            f, t = each
            stat, p = scipy.stats.wilcoxon(ranks[f], ranks[t])
            res = res.append(
                dict(
                    zip(res.columns, 
                        [key[0], key[1], k + "=" + str(f), k + "=" + str(t), src.utils.calculate_mrr(ranks[f]), src.utils.calculate_mrr(ranks[t]), p]))
                , ignore_index=True)

In [34]:
len(res[res.p < 0.05]), len(res[res.p >= 0.05])

(90, 90)

In [35]:
res[(res.MRR > res.MRR2) & (res.p < 0.05)]

,Subject,Task,Config,Config2,MRR,MRR2,p
1,BookKeeper v4.3.0,DIT,alpha=1,alpha=5,0.591286,0.584019,4.008216e-02
2,BookKeeper v4.3.0,DIT,alpha=1,alpha=auto,0.591286,0.586230,1.406231e-02
3,BookKeeper v4.3.0,DIT,alpha=2,alpha=5,0.593140,0.584019,1.082397e-02
4,BookKeeper v4.3.0,DIT,alpha=2,alpha=auto,0.593140,0.586230,4.398987e-03
7,BookKeeper v4.3.0,FLT,alpha=1,alpha=5,0.522396,0.516043,3.518887e-02
17,Mahout v0.10.0,DIT,alpha=5,alpha=auto,0.273479,0.265132,3.635855e-02
19,Mahout v0.10.0,FLT,alpha=1,alpha=5,0.653151,0.636244,1.800127e-03
33,OpenJPA v2.3.0,FLT,alpha=2,alpha=5,0.297471,0.296196,2.559760e-03
47,Pig v0.14.0,FLT,alpha=5,alpha=auto,0.432464,0.424027,2.765493e-02
60,ZooKeeper v3.5.0,DIT,alpha=1,alpha=2,0.325575,0.324924,2.808039e-02


In [36]:
res[(res.MRR < res.MRR2) & (res.p < 0.05)]

,Subject,Task,Config,Config2,MRR,MRR2,p
12,Mahout v0.10.0,DIT,alpha=1,alpha=2,0.259485,0.261637,5.983285e-03
13,Mahout v0.10.0,DIT,alpha=1,alpha=5,0.259485,0.273479,4.921763e-02
15,Mahout v0.10.0,DIT,alpha=2,alpha=5,0.261637,0.273479,4.782586e-05
16,Mahout v0.10.0,DIT,alpha=2,alpha=auto,0.261637,0.265132,1.727983e-02
23,Mahout v0.10.0,FLT,alpha=5,alpha=auto,0.636244,0.648777,9.885885e-04
31,OpenJPA v2.3.0,FLT,alpha=1,alpha=5,0.283422,0.296196,5.788439e-03
36,Pig v0.14.0,DIT,alpha=1,alpha=2,0.180605,0.197116,1.189699e-03
37,Pig v0.14.0,DIT,alpha=1,alpha=5,0.180605,0.202775,1.302695e-09
38,Pig v0.14.0,DIT,alpha=1,alpha=auto,0.180605,0.193631,4.495017e-05
39,Pig v0.14.0,DIT,alpha=2,alpha=5,0.197116,0.202775,6.741825e-04


In [37]:
t = res[(res.Config == "alpha=1") | (res.Config2 == "alpha=1")]
t

,Subject,Task,Config,Config2,MRR,MRR2,p
0,BookKeeper v4.3.0,DIT,alpha=1,alpha=2,0.591286,0.593140,6.924327e-01
1,BookKeeper v4.3.0,DIT,alpha=1,alpha=5,0.591286,0.584019,4.008216e-02
2,BookKeeper v4.3.0,DIT,alpha=1,alpha=auto,0.591286,0.586230,1.406231e-02
6,BookKeeper v4.3.0,FLT,alpha=1,alpha=2,0.522396,0.514261,1.975504e-01
7,BookKeeper v4.3.0,FLT,alpha=1,alpha=5,0.522396,0.516043,3.518887e-02
8,BookKeeper v4.3.0,FLT,alpha=1,alpha=auto,0.522396,0.515176,6.325219e-01
12,Mahout v0.10.0,DIT,alpha=1,alpha=2,0.259485,0.261637,5.983285e-03
13,Mahout v0.10.0,DIT,alpha=1,alpha=5,0.259485,0.273479,4.921763e-02
14,Mahout v0.10.0,DIT,alpha=1,alpha=auto,0.259485,0.265132,9.694895e-01
18,Mahout v0.10.0,FLT,alpha=1,alpha=2,0.653151,0.650913,8.064128e-02


In [38]:
len(t), len(t[t.MRR > t.MRR2]), len(t[t.p < 0.05])

(36, 17, 15)